In [15]:
from rdflib import Graph, URIRef, BNode, Literal, Namespace
from rdflib.namespace import RDF, RDFS

In [5]:
from pyfuseki import FusekiQuery, FusekiUpdate

In [6]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'thesaurus')

In [10]:
#DELETE
sparql_str = """DROP DEFAULT"""

update_result = fuseki_update.run_sparql(sparql_str)

In [12]:
objParser = {
    "authority": "Metodologia",
    "tokenLSCH": "sh85084414",
    "variant" : "Filosofia - Metodologia",
    "reciprocalAuthority": {"value": "Pesquisar", "uri": "http://id.loc.gov/authorities/subjects/sh85113021" },
    "narrowerAuthority" : [
        {"value": "Genealogia (Filosofia)", "uri": "http://id.loc.gov/authorities/subjects/sh94000124"},
        {"value": "Experiências mentais", "uri": "http://id.loc.gov/authorities/subjects/sh85050813"},
        {"value": "Performativo (Filosofia)", "uri": "http://id.loc.gov/authorities/subjects/sh85099817"},
        {"value": "Formalização (Filosofia)", "uri": "http://id.loc.gov/authorities/subjects/sh00002301"},
        {"value": "Abordagem interdisciplinar do conhecimento", "uri": "http://id.loc.gov/authorities/subjects/sh85067232"},
        {"value": "Análise (Filosofia)", "uri": "http://id.loc.gov/authorities/subjects/sh85004780"}, 
        {"value": "Método longitudinal", "uri": "http://id.loc.gov/authorities/subjects/sh85026813"},
        {"value": "Classificação das ciências", "uri": "http://id.loc.gov/authorities/subjects/sh85078296"},
        {"value": "Heurística", "uri": "http://id.loc.gov/authorities/subjects/sh85060556"}, 
        {"value": "Solução de problemas", "uri": "http://id.loc.gov/authorities/subjects/sh85107109"}
    ]
}

In [ ]:
for i in objParser.get("narrowerAuthority"):
    uri = URIRef(i.get('uri'))
    print(i.get('uri'))

In [13]:
def HasNarrowerAuthority(g, subject, MADSRDF):
    for i in objParser.get("narrowerAuthority"):
        uri = URIRef(i.get('uri'))
        value = Literal(i.get('value'), lang='pt')
        g.add((subject, MADSRDF.hasNarrowerAuthority, uri))

        g.add((uri, RDF.type, MADSRDF.Authority))
        g.add((uri, MADSRDF.authoritativeLabel, value))
    return g


In [16]:
def CreateSubject(objParser):

    MADSRDF = Namespace("http://www.loc.gov/mads/rdf/v1#")

    token = objParser.get("tokenLSCH")
    authority = objParser.get("authority")
    authorityLit = Literal(authority, lang='pt')
    variantlabel = objParser.get("variant")
    reciprocalAuthorityUri = URIRef(objParser.get('reciprocalAuthority').get('uri'))
    reciprocalAuthority = Literal(objParser.get('reciprocalAuthority').get('value'), lang='pt')

    g = Graph()
    g.bind("madsrdf", MADSRDF)
    g.bind("rdf", RDF)

    #authoritativeLabel
    subject = URIRef(f"https://bibliokeia.com/authorities/subjects/{token}")
    g.add((subject, RDF.type, MADSRDF.Authority))
    g.add((subject, RDF.type, MADSRDF.Topic))
    g.add((subject, MADSRDF.authoritativeLabel, authorityLit))
    elementListAuthority = BNode()
    g.add((subject, MADSRDF.elementList, elementListAuthority))
    elementAuthority = BNode()
    g.add((elementListAuthority, RDF.first, elementAuthority))
    g.add((elementAuthority, RDF.type, MADSRDF.TopicElement))
    g.add((elementAuthority, MADSRDF.elementValue, authorityLit))
    g.add((elementListAuthority, RDF.rest, RDF.nil))

    #variant
    variantBN = BNode()
    g.add((subject, MADSRDF.hasVariant, variantBN))
    g.add((variantBN, RDF.type, MADSRDF.ComplexSubject))
    g.add((variantBN, RDF.type, MADSRDF.Variant))
    g.add((variantBN, MADSRDF.variantLabel, Literal(variantlabel, lang='pt')))

    componentListBN = BNode()
    g.add((variantBN, MADSRDF.componentList, componentListBN))
    variantBN1 = BNode()
    g.add((componentListBN, RDF.first, variantBN1))
    g.add((variantBN1, RDF.type, MADSRDF.Topic))
    g.add((variantBN1, RDF.type, MADSRDF.Variant))
    #elementList
    elementListBN1 = BNode()
    g.add((variantBN1, MADSRDF.elementList, elementListBN1))
    elementBN1 = BNode()
    g.add((elementListBN1, RDF.first, elementBN1))
    g.add((elementBN1, RDF.type, MADSRDF.TopicElement))
    g.add((elementBN1, MADSRDF.elementValue, Literal("Filosofia", lang='pt')))
    g.add((elementListBN1, RDF.rest, RDF.nil))
    g.add((variantBN1, MADSRDF.variantLabel, Literal("Filosofia", lang='pt')))
    firstBN = BNode()
    g.add((componentListBN, RDF.rest, firstBN))

    variantBN2 = BNode()
    g.add((firstBN, RDF.first, variantBN2))
    g.add((variantBN2, RDF.type, MADSRDF.Topic))
    g.add((variantBN2, RDF.type, MADSRDF.Variant))

    #elementList
    elementListBN2 = BNode()
    g.add((variantBN2, MADSRDF.elementList, elementListBN2))
    elementBN2 = BNode()
    g.add((elementListBN2, RDF.first, elementBN2))
    g.add((elementBN2, RDF.type, MADSRDF.TopicElement))
    g.add((elementBN2, MADSRDF.elementValue, Literal("Metodologia", lang='pt')))
    g.add((elementListBN2, RDF.rest, RDF.nil))
    g.add((variantBN2, MADSRDF.variantLabel, Literal("Metodologia", lang='pt')))
    g.add((firstBN, RDF.rest, RDF.nil))

    #hasReciprocalAuthority
    g.add((subject, MADSRDF.hasReciprocalAuthority, reciprocalAuthorityUri))
    g.add((reciprocalAuthorityUri, RDF.type, MADSRDF.Authority))
    g.add((reciprocalAuthorityUri, MADSRDF.authoritativeLabel, reciprocalAuthority))

    #hasNarrowerAuthority
    g = HasNarrowerAuthority(g, subject, MADSRDF)

    #g.serialize('TESTE.nt')
    return g

g = CreateSubject(objParser)



In [21]:
def Make_Graph(nt, token ):
    G1 = """PREFIX bk: <https://bibliokeia.com/authorities/subjects/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {
        GRAPH bk:"""

    G2 = " {"

    G3 = """}
    }"""

    G = G1+str(token)+" { \n"+nt+"}}"

    return G

In [19]:
nt = g.serialize(format='nt')

In [22]:
G = Make_Graph(nt, "sh85084414")

In [25]:
r = fuseki_update.run_sparql(G)

TypeError: Cannot instantiate typing.Union